In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [3]:
url = 'https://up.codes/s/equivalent-pipe-lengths-of-valves-and-fittings'

html = requests.get(url)

In [4]:
soup = BeautifulSoup(html.content, 'html.parser')

table = soup.find('table', class_='u-mt-2')

rows = table.find_all('tr')

diameters = {'(in)':[], '(mm)':[]}

for row in rows:

    cells = row.find_all('th')

    for cell in cells:
        cell_text = cell.get_text()
        if 'in.' in cell_text:
            cell_text = cell_text.replace('in.', '')
            diameters['(in)'].append(cell_text)

        elif 'mm)' in cell_text:
            cell_text = cell_text.replace('mm', '')
            cell_text = cell_text.replace('(','')
            cell_text = cell_text.replace(')','')
            diameters['(mm)'].append(cell_text)

        else:
            continue

In [7]:
elements = []

tbody = table.find('tbody')

rows = tbody.find_all('tr')

for element in rows:
    row_of_element = element.find_all('td')

    for equivalent in row_of_element:
        elements.append(equivalent.get_text())

print(elements)

['45°elbow', '—', '1 (0.3)', '1 (0.3)', '1 (0.3)', '2 (0.6)', '2 (0.6)', '3 (0.9)', '3 (0.9)', '3 (0.9)', '4 (1.2)', '5 (1.5)', '7 (2.1)', '9 (2.7)', '11 (3.3)', '13 (4)', '90°standard elbow', '1 (0.3)', '2 (0.6)', '2 (0.6)', '3 (0.9)', '4 (1.2)', '5 (1.5)', '6 (1.8)', '7 (2.1)', '8 (2.4)', '10 (3.0)', '12 (3.7)', '14 (4.3)', '18 (5.5)', '22 (6.7)', '27 (8.2)', '90°long-turn elbow', '0.5 (0.2)', '1 (0.3)', '2 (0.6)', '2 (0.6)', '2 (0.6)', '3 (0.9)', '4 (1.2)', '5 (1.5)', '5 (1.5)', '6 (1.8)', '8 (2.4)', '9 (2.7)', '13 (4)', '16 (4.9)', '18 (5.5)', 'Tee or cross (flow turned 90°)', '3 (0.9)', '4 (1.2)', '5 (1.5)', '6 (1.8)', '8 (2.4)', '10 (3.0)', '12 (3.7)', '15 (4.6)', '17 (5.2)', '20 (6.1)', '25 (7.6)', '30 (9.1)', '35 (10.7)', '50 (15.2)', '60 (18.3)', 'Butterfly valve', '—', '—', '—', '—', '—', '6 (1.8)', '7 (2.1)', '10 (3.0)', '—', '12 (3.7)', '9 (2.7)', '10 (3.0)', '12 (3.7)', '19 (5.8)', '21 (6.4)', 'Gate valve', '—', '—', '—', '—', '—', '1 (0.3)', '1 (0.3)', '1 (0.3)', '1 (0.3)

In [8]:
del diameters['(in)'][-1]
del diameters['(in)'][-1]
del diameters['(mm)'][-1]
del diameters['(mm)'][-1]

print(diameters)

{'(in)': ['1/2 ', '3/4 ', '1 ', '11/4 ', '11/2 ', '2 ', '21/2 ', '3 ', '31/2 ', '4 ', '5 '], '(mm)': ['15 ', '20 ', '25 ', '32 ', '40 ', '50 ', '65 ', '80 ', '90 ', '100 ', '125 ']}


In [9]:
table_content_imperial_dict = {}
table_content_metric_dict = {}

for element in elements:

    if '(' not in element and ')' not in element and '—' not in element:

        table_content_imperial_dict[element] = []
        table_content_metric_dict[element] = []
        last_key = element

    elif '—' in element:

        table_content_imperial_dict[last_key].append(None)
        table_content_metric_dict[last_key].append(None)

    else:
        first_string_index = element.find('(')
        second_string_index = element.find(')')
        table_content_imperial_dict[last_key].append(element[:first_string_index])
        table_content_metric_dict[last_key].append(element[first_string_index + 1:second_string_index])


In [7]:
print(table_content_imperial_dict)

{'45°elbow': [None, '1 ', '1 ', '1 ', '2 ', '2 ', '3 ', '3 ', '3 ', '4 ', '5 ', '7 ', '9 ', '11 ', '13 '], '90°standard elbow': ['1 ', '2 ', '2 ', '3 ', '4 ', '5 ', '6 ', '7 ', '8 ', '10 ', '12 ', '14 ', '18 ', '22 ', '27 '], '90°long-turn elbow': ['0.5 ', '1 ', '2 ', '2 ', '2 ', '3 ', '4 ', '5 ', '5 ', '6 ', '8 ', '9 ', '13 ', '16 ', '18 ', 'Tee or cross ', '3 ', '4 ', '5 ', '6 ', '8 ', '10 ', '12 ', '15 ', '17 ', '20 ', '25 ', '30 ', '35 ', '50 ', '60 '], 'Butterfly valve': [None, None, None, None, None, '6 ', '7 ', '10 ', None, '12 ', '9 ', '10 ', '12 ', '19 ', '21 '], 'Gate valve': [None, None, None, None, None, '1 ', '1 ', '1 ', '1 ', '2 ', '2 ', '3 ', '4 ', '5 ', '6 '], 'Vane type flow switch': [], '': ['6 ', '9 ', '10 ', '14 ', '17 ', '22 ', None, '30 ', None, '16 ', '22 ', '29 ', '36 '], 'Swing check*': [None, None, '5 ', '7 ', '9 ', '11 ', '14 ', '16 ', '19 ', '22 ', '27 ', '32 ', '45 ', '55 ', '65 ']}


In [10]:
for key in list(table_content_imperial_dict.keys()):
    try:
        del table_content_imperial_dict[key][-1]
        del table_content_imperial_dict[key][-1]
        del table_content_metric_dict[key][-1]
        del table_content_metric_dict[key][-1]
    except:
        pass

In [95]:
print(table_content_imperial_dict)

{'45°elbow': [None, '1 ', '1 ', '1 ', '2 ', '2 ', '3 ', '3 ', '3 ', '4 ', '5 ', '7 ', '9 '], '90°standard elbow': ['1 ', '2 ', '2 ', '3 ', '4 ', '5 ', '6 ', '7 ', '8 ', '10 ', '12 ', '14 ', '18 '], '90°long-turn elbow': ['0.5 ', '1 ', '2 ', '2 ', '2 ', '3 ', '4 ', '5 ', '5 ', '6 ', '8 ', '9 ', '13 ', '16 ', '18 ', 'Tee or cross ', '3 ', '4 ', '5 ', '6 ', '8 ', '10 ', '12 ', '15 ', '17 ', '20 ', '25 ', '30 ', '35 '], 'Butterfly valve': [None, None, None, None, None, '6 ', '7 ', '10 ', None, '12 ', '9 ', '10 ', '12 '], 'Gate valve': [None, None, None, None, None, '1 ', '1 ', '1 ', '1 ', '2 ', '2 ', '3 ', '4 '], 'Vane type flow switch': [], '': ['6 ', '9 ', '10 ', '14 ', '17 ', '22 ', None, '30 ', None, '16 ', '22 '], 'Swing check*': [None, None, '5 ', '7 ', '9 ', '11 ', '14 ', '16 ', '19 ', '22 ', '27 ', '32 ', '45 ']}


In [11]:
imperial_data = {}

diameter_keys_imperial = list(diameters['(in)'])

fittings_to_remove = []

for i, diameter in enumerate(diameter_keys_imperial):
    imperial_data[diameter] = {}
    for fitting in table_content_imperial_dict.keys():
        try:
            imperial_data[diameter][fitting] = table_content_imperial_dict[fitting][i]
        except KeyError:
            pass
        except IndexError:
            if fitting not in fittings_to_remove and fitting != '':
                fittings_to_remove.append(fitting)

for fitting in fittings_to_remove:
    if fitting in table_content_imperial_dict:
        del table_content_imperial_dict[fitting]

fittings_imperial_df = pd.DataFrame(imperial_data)

fittings_imperial_df = fittings_imperial_df.drop(fittings_imperial_df.index[-2])

fittings_imperial_df

,1/2,3/4,1,11/4,11/2,2,21/2,3,31/2,4,5
45°elbow,None,1,1,1,2,2,3,3,3,4,5
90°standard elbow,1,2,2,3,4,5,6,7,8,10,12
90°long-turn elbow,0.5,1,2,2,2,3,4,5,5,6,8
Butterfly valve,None,None,None,None,None,6,7,10,None,12,9
Gate valve,None,None,None,None,None,1,1,1,1,2,2
Swing check*,None,None,5,7,9,11,14,16,19,22,27


In [14]:
metric_data = {}

diameter_keys_metric = list(diameters['(mm)'])

fittings_to_remove = []

for i, diameter in enumerate(diameter_keys_metric):
    metric_data[diameter] = {}
    for fitting in table_content_metric_dict.keys():
        try:
            metric_data[diameter][fitting] = table_content_metric_dict[fitting][i]
        except KeyError:
            pass
        except IndexError:
            if fitting not in fittings_to_remove and fitting != '':
                fittings_to_remove.append(fitting)

for fitting in fittings_to_remove:
    if fitting in table_content_metric_dict:
        del table_content_metric_dict[fitting]

fittings_metric_df = pd.DataFrame(metric_data)

fittings_metric_df = fittings_metric_df.drop(fittings_metric_df.index[-2])

fittings_metric_df


,15,20,25,32,40,50,65,80,90,100,125
45°elbow,None,0.3,0.3,0.3,0.6,0.6,0.9,0.9,0.9,1.2,1.5
90°standard elbow,0.3,0.6,0.6,0.9,1.2,1.5,1.8,2.1,2.4,3.0,3.7
90°long-turn elbow,0.2,0.3,0.6,0.6,0.6,0.9,1.2,1.5,1.5,1.8,2.4
Butterfly valve,None,None,None,None,None,1.8,2.1,3.0,None,3.7,2.7
Gate valve,None,None,None,None,None,0.3,0.3,0.3,0.3,0.6,0.6
Swing check*,None,None,1.5,2.1,2.7,3.3,4.3,4.9,5.8,6.7,8.2


In [12]:
fittings_imperial_df.to_csv(path_or_buf=r'C:\Users\jc-14\Documents\Python\Hydraulic Calculation\Modules\Materials_data\fittings_imperial.csv',
                            header = diameters['(in)'], index=True, na_rep='None')

In [15]:
fittings_metric_df.to_csv(path_or_buf=r'C:\Users\jc-14\Documents\Python\Hydraulic Calculation\Modules\Materials_data\fittings_metric.csv',
                            header = diameters['(mm)'], index=True, na_rep='None')